In [1]:
from utils import image_to_text
import spacy
from matplotlib import pyplot as plt
from model import Net
import torch
from dataset import TrainDataset
from model import Net
from torch.utils.data import DataLoader
import torch.optim as optim
from tqdm import tqdm

In [2]:
device = torch.device('cpu')
epochs = 50

In [3]:
train_dataset = TrainDataset()
train_dataloader = DataLoader(train_dataset, batch_size = 16, shuffle=True)

In [4]:
batch = next(iter(train_dataloader))
input_ids = batch[0].to(device)
labels    = batch[1].float().to(device)
text      = batch[2]

In [5]:
model = Net(len(train_dataset.VOCAB)+1).to(device)
model.eval()

Building NN...


Net(
  (l1): Embedding(72, 16, padding_idx=0)
  (l2): LSTM(16, 128, num_layers=2, bidirectional=True)
  (l3): Linear(in_features=256, out_features=5, bias=True)
)

In [6]:
# Optimizer and Loss Function
optimizer    = optim.Adam(model.parameters(), lr = 0.0001)
lossFunction = torch.nn.CrossEntropyLoss()
scheduler    = optim.lr_scheduler.StepLR(optimizer, 1000)

In [8]:
best_loss = float('INF')
model.train()
for epoch in range(epochs):
    print("Epoch", epoch+1, ":")
    
    loss_train = 0.0
    n_batches  = 0
    errors = 0
    # -------- TRAINING ----------
    loop = tqdm(train_dataloader, leave = True)
    for batch in loop:
        # check for invalid entries
        n_batches += 1
        # reset
        optimizer.zero_grad()
        
        # pull all tensor batches required for training
        input_ids = batch[0].to(device)
        labels    = batch[1].float().to(device)
        
        # pass data through the model
        outputs    = model(input_ids)

        loss       = lossFunction(outputs, labels)
        loss_train += loss.item()
        # backpropagate
        loss.backward()
        # update parameters
        optimizer.step()
    
    if(loss_train < best_loss):
        best_loss = loss_train
        torch.save(model.state_dict(), 'models/best-model-parameters.pt')
    print("Epoch Done! Train Loss: ", loss_train/n_batches)
    scheduler.step()

  0%|          | 0/40 [00:00<?, ?it/s]

Epoch 1 :


  0%|          | 0/40 [00:00<?, ?it/s]

Epoch Done! Train Loss:  991.8045715332031
Epoch 2 :


  0%|          | 0/40 [00:00<?, ?it/s]

Epoch Done! Train Loss:  923.0705627441406
Epoch 3 :


  0%|          | 0/40 [00:01<?, ?it/s]


KeyboardInterrupt: 